In [1]:
!pip install -q -U kaggle_environments

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 27.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.8/953.8 kB 34.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 840.2/840.2 kB 32.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 11.0 MB/s eta 0:00:00


In [5]:
import random

from kaggle_environments import make

Определим 13 различных стратегий для агентов:

In [6]:


# Агент, который всегда выбирает камень
def rock_agent(observation, configuration):
    return 0

# Агент, который всегда выбирает ножницы
def scissors_agent(observation, configuration):
    return 1

# Агент, который всегда выбирает бумагу
def paper_agent(observation, configuration):
    return 2

# Агент, который выбирает случайный ход
def random_agent(observation, configuration):
    return random.randint(0, 2)

# Агент, который повторяет последний ход противника
def copy_opponent_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        return observation.lastOpponentAction

# Агент, который циклически выбирает камень, ножницы, бумагу
def cycle_agent(observation, configuration):
    return observation.step % 3

# Агент, который выбирает ход, побеждающий последний ход противника
def beat_last_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        return (observation.lastOpponentAction + 1) % 3

# Агент, который выбирает ход, проигрывающий последний ход противника
def lose_last_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        return (observation.lastOpponentAction + 2) % 3

# Агент, который выбирает ход, который чаще всего побеждал
def most_winning_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        actions = [0, 1, 2]
        wins = [(observation.lastOpponentAction + 1) % 3,
                observation.lastOpponentAction,
                 (observation.lastOpponentAction + 2) % 3]
        return wins[random.choice(actions)]

# Агент, который выбирает ход, который чаще всего проигрывал
def most_losing_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        actions = [0, 1, 2]
        losses = [(observation.lastOpponentAction + 2) % 3,
                  observation.lastOpponentAction,
                   (observation.lastOpponentAction + 1) % 3]
        return losses[random.choice(actions)]

# Агент, который выбирает ход, который чаще всего ничья
def most_drawing_agent(observation, configuration):
    if observation.step == 0:
        return random.randint(0, 2)
    else:
        return observation.lastOpponentAction

# Агент, который выбирает случайный ход, но с предпочтением камня
def biased_rock_agent(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.5, 0.25, 0.25])[0]

# Агент, который выбирает случайный ход, но с предпочтением ножниц
def biased_scissors_agent(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.25, 0.5, 0.25])[0]

# Агент, который выбирает случайный ход, но с предпочтением бумаги
def biased_paper_agent(observation, configuration):
    return random.choices([0, 1, 2], weights=[0.25, 0.25, 0.5])[0]

Запустим турнир между агентами:

In [7]:
# Создаем окружение для игры
env = make("rps", configuration={"episodeSteps": 100})

# Определяем список агентов
agents = [rock_agent, scissors_agent, paper_agent, random_agent,
          copy_opponent_agent, cycle_agent, beat_last_agent,
          lose_last_agent, most_winning_agent, most_losing_agent,
          most_drawing_agent, biased_rock_agent, biased_scissors_agent,
          biased_paper_agent
]

# Инициализируем счетчик побед для каждого агента
victories = {f"Agent {i+1}": 0 for i in range(len(agents))}

# Запускаем турнир
results = {}
for i, agent1 in enumerate(agents):
    for j, agent2 in enumerate(agents):
        if i < j:
            # Запускаем игру между двумя агентами
            env.reset()
            env.run([agent1, agent2])
            outcome = env.state[0]['reward'] - env.state[1]['reward']
            if outcome > 0:
                winner = f"Agent {i+1}"
                victories[winner] += 1
            elif outcome < 0:
                winner = f"Agent {j+1}"
                victories[winner] += 1
            else:
                winner = "Draw"
            results[f"Agent {i+1} vs Agent {j+1}"] = winner

# Выводим результаты матчей
for match, winner in results.items():
    print(f"{match}: {winner}")

# Определяем агента с наибольшим количеством побед
best_agent = max(victories, key=victories.get)
print(f"\nЛучший агент: {best_agent} с {victories[best_agent]} победами")

Agent 1 vs Agent 2: Agent 2
Agent 1 vs Agent 3: Agent 1
Agent 1 vs Agent 4: Draw
Agent 1 vs Agent 5: Draw
Agent 1 vs Agent 6: Draw
Agent 1 vs Agent 7: Agent 7
Agent 1 vs Agent 8: Agent 1
Agent 1 vs Agent 9: Draw
Agent 1 vs Agent 10: Draw
Agent 1 vs Agent 11: Draw
Agent 1 vs Agent 12: Draw
Agent 1 vs Agent 13: Agent 13
Agent 1 vs Agent 14: Draw
Agent 2 vs Agent 3: Agent 3
Agent 2 vs Agent 4: Draw
Agent 2 vs Agent 5: Draw
Agent 2 vs Agent 6: Draw
Agent 2 vs Agent 7: Agent 7
Agent 2 vs Agent 8: Agent 2
Agent 2 vs Agent 9: Draw
Agent 2 vs Agent 10: Draw
Agent 2 vs Agent 11: Draw
Agent 2 vs Agent 12: Agent 2
Agent 2 vs Agent 13: Draw
Agent 2 vs Agent 14: Draw
Agent 3 vs Agent 4: Draw
Agent 3 vs Agent 5: Draw
Agent 3 vs Agent 6: Draw
Agent 3 vs Agent 7: Agent 7
Agent 3 vs Agent 8: Agent 3
Agent 3 vs Agent 9: Agent 3
Agent 3 vs Agent 10: Draw
Agent 3 vs Agent 11: Draw
Agent 3 vs Agent 12: Agent 12
Agent 3 vs Agent 13: Draw
Agent 3 vs Agent 14: Draw
Agent 4 vs Agent 5: Draw
Agent 4 vs Agent 6:

Этот код создает 13 агентов с различными стратегиями и проводит турнир, определяя победителя в каждой паре.